<a href="https://colab.research.google.com/github/YapingWu/GoogleColab/blob/main/keras/WordLstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word-Level LSTM实现文本生成

## 准备工作

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, TimeDistributed
from keras import optimizers
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import time
import pickle

In [18]:
data_name = 'myspace'
max_lengths = {
    'myspace': 35,
    'phpbb': 21,
    'rockyou': 41,
}
SEED = 7

In [10]:
word_list_file = '{}{}.txt'.format('/content/', data_name)
tokenizer_file = '{}{}.pkl'.format('/content/', data_name)
print(word_list_file)
print(tokenizer_file)
with open(tokenizer_file, 'rb') as file:
  tokenizer = pickle.load(file)
vocab_size = len(tokenizer.word_index) + 1
max_length = max_lengths[data_name]

/content/myspace.txt
/content/myspace.pkl


## 加载数据

In [12]:
word_list = pd.read_csv(word_list_file)['grammar']
word_list = word_list.values.tolist()

In [16]:
# 将编码后的密码转换为整数序列
print("将编码后的密码转换为（整数）序列")
sequences = list()
for line in word_list:  # 'L8 D1 '
    line += '<END>'
    # 将文本转换为（整数）序列
    encoded = tokenizer.texts_to_sequences([line])[0]
    # 过滤掉长度大于 MAX_SEQ_LEN 的序列
    if len(encoded) > max_length:
        continue
    for i in range(1, len(encoded) + 1):
        sequence = encoded[:i]
        sequences.append(sequence)

print('Total Sequences: %d' % len(sequences))

# pad input sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, max_length, padding='pre')  # 左边填充0
print('Max Sequence Length: %d' % max_length)

# 创建输入输出
print("创建LSTM模型的输入输出")
sequences = np.array(sequences)
x, y = sequences[:, :-1], sequences[:, -1]
print("X Shape: %s, y Shape: %s" % (x.shape, y.shape))
y = to_categorical(y, num_classes=vocab_size)  # 对输出进行one-hot编码

将编码后的密码转换为（整数）序列
Total Sequences: 8976
Max Sequence Length: 35
创建LSTM模型的输入输出
X Shape: (8976, 34), y Shape: (8976,)


### 划分训练集、验证集和测试集

In [19]:
def split_xy(x, y):

    ratio = 0.6  # 训练集比例
    if len(x) > 100000:
        ratio = 0.9

    print("划分训练集、验证集和测试集")
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1 - ratio, random_state=SEED)
    x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=SEED)

    print("x_train Shape: %s, y_train Shape: %s" % (x_train.shape, y_train.shape))
    print("x_val Shape: %s, y_val Shape: %s" % (x_val.shape, y_val.shape))
    print("x_test Shape: %s, y_test Shape: %s" % (x_test.shape, y_test.shape))

    return x_train, x_val, x_test, y_train, y_val, y_test

x_train, x_val, x_test, y_train, y_val, y_test = split_xy(x, y)

划分训练集、验证集和测试集
x_train Shape: (5385, 34), y_train Shape: (5385, 73)
x_val Shape: (1795, 34), y_val Shape: (1795, 73)
x_test Shape: (1796, 34), y_test Shape: (1796, 73)


## 创建和训练模型

In [40]:
cur_time = time.strftime("%y%m%d%H%M%S", time.localtime())
log_name = '{}{}{}'.format('./logs/', data_name, cur_time)
model_file = '{}{}.h5'.format('./model/', data_name)

epochs = 30
batch_size = 128
lstm_layers = [[256, True], [256, True], [256, False]]
fully_layers = [512]

### 创建模型

In [41]:
print('{:*^106}'.format('创建LSTM模型'))
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=max_length - 1))

for hidden_size, rs in lstm_layers:
    model.add(LSTM(hidden_size, return_sequences=rs))

for hidden_size in fully_layers:
    model.add(Dense(units=hidden_size, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

model.summary()

adam = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

*************************************************创建LSTM模型*************************************************
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 34, 10)            730       
_________________________________________________________________
lstm_21 (LSTM)               (None, 34, 256)           273408    
_________________________________________________________________
lstm_22 (LSTM)               (None, 34, 256)           525312    
_________________________________________________________________
lstm_23 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_14 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_15 (Dense)             (None, 73)                37449     
Total params:

### 训练模型

In [35]:
tensorboard = TensorBoard(log_dir=log_name)

print('{:*^106}'.format('开始训练LSTM模型'))
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          epochs=epochs,
          batch_size=batch_size,
          verbose=1,
          callbacks=[tensorboard])
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print("Model Accuracy on test: %.2f%%, Loss: %.2f" % (accuracy * 100, loss))
print('{:*^106}'.format('完成训练LSTM模型'))

************************************************开始训练LSTM模型************************************************
Epoch 1/30
43/43 [==============================] - 45s 1s/step - loss: 2.6538 - accuracy: 0.1783 - val_loss: 2.6287 - val_accuracy: 0.1894
Epoch 2/30
43/43 [==============================] - 44s 1s/step - loss: 2.6612 - accuracy: 0.1835 - val_loss: 2.6254 - val_accuracy: 0.1894
Epoch 3/30
43/43 [==============================] - 44s 1s/step - loss: 2.6588 - accuracy: 0.1842 - val_loss: 2.6166 - val_accuracy: 0.1727
Epoch 4/30
43/43 [==============================] - 44s 1s/step - loss: 2.6416 - accuracy: 0.1853 - val_loss: 2.6132 - val_accuracy: 0.1727
Epoch 5/30
43/43 [==============================] - 44s 1s/step - loss: 2.5861 - accuracy: 0.2178 - val_loss: 2.5450 - val_accuracy: 0.2490
Epoch 6/30
43/43 [==============================] - 44s 1s/step - loss: 2.5352 - accuracy: 0.2494 - val_loss: 2.4681 - val_accuracy: 0.2696
Epoch 7/30
43/43 [==============================] - 4

### 保存模型

In [ ]:
print("保存模型：%s" % model_file)
model.save(model_file)

# 其他命令

In [1]:
!unzip '/content/wordlist.zip'

Archive:  /content/wordlist.zip
  inflating: myspace.txt             
  inflating: phpbb.txt               
  inflating: rockyou.txt             


In [2]:
!unzip '/content/tokenizer.zip'

Archive:  /content/tokenizer.zip
  inflating: phpbb.pkl               
  inflating: rockyou.pkl             
  inflating: myspace.pkl             


In [36]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir '/content/logs/myspace210301071211'